In [1]:
# загрузка библиотек
import pandas as pd
import urllib.parse as up
import psycopg2

## 1. Подключение к базе данных

In [2]:
# загрузка конфигурации доступа к базе данных в облаке
with open('DATABASE_URL.txt', 'r') as f:
    DATABASE_URL = f.read()

In [3]:
# разбивка на компоненты
url = up.urlparse(DATABASE_URL)

In [4]:
# создание подключения к базе данны
connect = psycopg2.connect(database=url.path[1:],
                           user=url.username,
                           password=url.password,
                           host=url.hostname,
                           port=url.port)

In [5]:
# создание объекта курсора подключения к базе данных
cursor = connect.cursor()

## 2. Создание схемы Bl model - Storage Layer

![solution_architecture](../Module01/images/solution_architecture.jpg)

In [6]:
# удаление схемы 'Bl model' если она существует 
#cursor.execute("DROP SCHEMA blm CASCADE;")
#connect.commit() 

In [7]:
# создание схемы 'Bl model' - Storage Layer
cursor.execute("CREATE SCHEMA blm;")
connect.commit()  

![logical_model](images/logical_model.png)

### 2.1 Создание таблицы сегменты - 'Segments'

In [8]:
# удаление таблицы 'segment' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.segments CASCADE;")
connect.commit()  

In [9]:
# создание таблицы сегменты - 'segments'
cursor.execute("CREATE TABLE blm.segments( \
               Segment_ID INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
               Segment VARCHAR(11) UNIQUE NOT NULL);")
connect.commit() 

### 2.2 Создание таблицы покупатели - 'Customers'

In [10]:
# удаление таблицы 'customers' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.customers CASCADE;")
connect.commit()  

In [11]:
# создание таблицы покупатели - 'customers'
cursor.execute("CREATE TABLE blm.customers( \
               Customer_ID VARCHAR(8) PRIMARY KEY, \
               Customer_Name VARCHAR(22) NOT NULL, \
               Segment_ID INTEGER REFERENCES blm.segments (segment_id));")
connect.commit() 

### 2.3 Создание таблицы класс доставки - 'Ship Modes'

In [12]:
# удаление таблицы 'ship_modes' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.ship_modes CASCADE;")
connect.commit()  

In [13]:
# создание таблицы класс доставки - 'ship_modes'
cursor.execute("CREATE TABLE blm.ship_modes( \
               Ship_mode_ID INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
               Ship_mode VARCHAR(14) UNIQUE NOT NULL);")
connect.commit() 

### 2.4 Создание таблицы категории  - 'Categorys'

In [14]:
# удаление таблицы 'categorys' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.categorys CASCADE;")
connect.commit()  

In [15]:
# создание таблицы категории категории - 'categorys'
cursor.execute("CREATE TABLE blm.categorys( \
               Sub_category_ID INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
               Sub_category VARCHAR(11) UNIQUE NOT NULL, \
               Category VARCHAR(15) NOT NULL);")
connect.commit() 

### 2.5 Создание таблицы категории продуктов - 'Products'

In [16]:
# удаление таблицы 'products' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.products CASCADE;")
connect.commit()  

In [17]:
# создание таблицы класс доставки - 'ship_modes'
cursor.execute("CREATE TABLE blm.products( \
               Row_ID INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
               Product_ID VARCHAR(15) NOT NULL, \
               Sub_category_ID INTEGER REFERENCES blm.categorys (Sub_category_ID), \
               Product_Name VARCHAR(127) NOT NULL);")
connect.commit() 

### 2.6 Создание таблицы регионы - 'Regions'

In [18]:
# удаление таблицы 'regions' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.regions CASCADE;")
connect.commit()  

In [19]:
# создание таблицы регионы - 'regions'
cursor.execute("CREATE TABLE blm.regions( \
               Region_ID INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
               Region VARCHAR(7) UNIQUE NOT NULL);")
connect.commit() 

### 2.7 Создание таблицы адреса - 'Geos'

In [20]:
# удаление таблицы 'geos' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.geos CASCADE;")
connect.commit()  

In [21]:
# создание таблицы адреса - 'geos'
cursor.execute("CREATE TABLE blm.geos( \
               Geo_ID INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
               Country VARCHAR(13) NOT NULL, \
               State VARCHAR(20) NOT NULL, \
               Region_ID INTEGER REFERENCES blm.regions (Region_ID), \
               City VARCHAR(17) NOT NULL, \
               Postal_Code INTEGER NOT NULL);")
connect.commit() 

### 2.8 Создание таблицы менеджеры - 'Persons'

In [22]:
# удаление таблицы 'persons' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.persons CASCADE;")
connect.commit()  

In [23]:
# создание таблицы менеджеры - 'persons'
cursor.execute("CREATE TABLE blm.persons( \
               Person_ID INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
               Person VARCHAR(17) NOT NULL, \
               Region_ID INTEGER REFERENCES blm.regions (Region_ID));")
connect.commit() 

### 2.9 Создание таблицы заказы - 'Orders'

In [24]:
# удаление таблицы 'orders' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.orders CASCADE;")
connect.commit()  

In [25]:
# создание таблицы заказы - 'orders'
cursor.execute("CREATE TABLE blm.orders( \
               Ord_ID INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
               Order_ID VARCHAR(14) NOT NULL, \
               Order_Date DATE NOT NULL, \
               Ship_Date DATE NOT NULL, \
               Ship_mode_ID INTEGER REFERENCES blm.ship_modes (Ship_mode_ID), \
               Customer_ID VARCHAR(8) REFERENCES blm.customers (Customer_ID), \
               Geo_ID INTEGER REFERENCES blm.geos (Geo_ID), \
               Row_ID INTEGER REFERENCES blm.products (Row_ID), \
               Sales NUMERIC(9,4) NOT NULL, \
               Quantity INTEGER NOT NULL, \
               Discount NUMERIC(4,2) NOT NULL, \
               Profit NUMERIC(21,16) NOT NULL, \
               Returned VARCHAR(3) NOT NULL, \
               Person_ID INTEGER REFERENCES blm.persons (Person_ID));")
connect.commit() 

In [26]:
# вывод наименований таблиц схемы 'Bl model'
cursor.execute("SELECT table_name FROM information_schema.tables \
             WHERE table_schema = 'blm';")
cursor.fetchall()

[('geos',),
 ('products',),
 ('orders',),
 ('persons',),
 ('segments',),
 ('categorys',),
 ('ship_modes',),
 ('regions',),
 ('customers',)]

## 3. Запись данных

### 3.1. Таблица категории - 'Categorys'

In [27]:
# данные для записи в таблицу категории - 'categorys'
cursor.execute("SELECT DISTINCT(SubCategory), Category FROM stg.orders;")
cursor.fetchall()

[('Paper', 'Office Supplies'),
 ('Copiers', 'Technology'),
 ('Furnishings', 'Furniture'),
 ('Appliances', 'Office Supplies'),
 ('Accessories', 'Technology'),
 ('Labels', 'Office Supplies'),
 ('Binders', 'Office Supplies'),
 ('Chairs', 'Furniture'),
 ('Supplies', 'Office Supplies'),
 ('Tables', 'Furniture'),
 ('Envelopes', 'Office Supplies'),
 ('Phones', 'Technology'),
 ('Bookcases', 'Furniture'),
 ('Art', 'Office Supplies'),
 ('Fasteners', 'Office Supplies'),
 ('Machines', 'Technology'),
 ('Storage', 'Office Supplies')]

In [28]:
# запись данных в таблицу продукты - 'categorys'
cursor.execute("INSERT INTO blm.categorys( \
                    Sub_category, Category) \
                    SELECT DISTINCT(SubCategory), Category FROM stg.orders;")
connect.commit() 

In [29]:
# проверка записи данных в таблицу 'categorys'
cursor.execute("SELECT * FROM blm.categorys;")
cursor.fetchall()

[(1, 'Paper', 'Office Supplies'),
 (2, 'Copiers', 'Technology'),
 (3, 'Furnishings', 'Furniture'),
 (4, 'Appliances', 'Office Supplies'),
 (5, 'Accessories', 'Technology'),
 (6, 'Labels', 'Office Supplies'),
 (7, 'Binders', 'Office Supplies'),
 (8, 'Chairs', 'Furniture'),
 (9, 'Supplies', 'Office Supplies'),
 (10, 'Tables', 'Furniture'),
 (11, 'Envelopes', 'Office Supplies'),
 (12, 'Phones', 'Technology'),
 (13, 'Bookcases', 'Furniture'),
 (14, 'Art', 'Office Supplies'),
 (15, 'Fasteners', 'Office Supplies'),
 (16, 'Machines', 'Technology'),
 (17, 'Storage', 'Office Supplies')]

### 3.2 Таблица категории продуктов - 'Products'

In [30]:
# проверка уникальности Product_ID
cursor.execute("SELECT Product_ID, Product_Name \
                FROM stg.orders \
                WHERE Product_ID IN (SELECT Product_ID FROM \
                (SELECT Product_ID, COUNT(DISTINCT(Product_Name)) \
                FROM stg.orders \
                GROUP BY Product_ID \
                HAVING COUNT(DISTINCT(Product_Name))>1) t) \
                GROUP BY Product_ID, Product_Name \
                ORDER BY Product_ID;")
cursor.fetchmany(10)

[('FUR-BO-10002213', 'Sauder Forest Hills Library, Woodland Oak Finish'),
 ('FUR-BO-10002213', 'DMI Eclipse Executive Suite Bookcases'),
 ('FUR-CH-10001146', "Global Value Mid-Back Manager's Chair, Gray"),
 ('FUR-CH-10001146', 'Global Task Chair, Black'),
 ('FUR-FU-10001473', 'DAX Wood Document Frame'),
 ('FUR-FU-10001473', 'Eldon Executive Woodline II Desk Accessories, Mahogany'),
 ('FUR-FU-10004017', 'Executive Impressions 13" Chairman Wall Clock'),
 ('FUR-FU-10004017',
  'Tenex Contemporary Contur Chairmats for Low and Medium Pile Carpet, Computer, 39" x 49"'),
 ('FUR-FU-10004091', 'Howard Miller 13" Diameter Goldtone Round Wall Clock'),
 ('FUR-FU-10004091', 'Eldon 200 Class Desk Accessories, Black')]

Одному и тому же **Product_ID** соответствует несколько наименований продуктов - **Product_Name**

In [31]:
# данные для записи в таблицу продукты - 'products'
cursor.execute("SELECT DISTINCT ON (Product_ID, Product_Name) Product_ID, Product_Name, Sub_category_ID \
                FROM stg.orders as ord \
                INNER JOIN blm.categorys as cat on ord.SubCategory = cat.Sub_category;")
cursor.fetchmany(3)

[('FUR-BO-10000112', 'Bush Birmingham Collection Bookcase, Dark Cherry', 13),
 ('FUR-BO-10000330',
  'Sauder Camden County Barrister Bookcase, Planked Cherry Finish',
  13),
 ('FUR-BO-10000362', 'Sauder Inglewood Library Bookcases', 13)]

In [32]:
# запись данных в таблицу продукты - 'products'
cursor.execute("INSERT INTO blm.products( \
                Product_ID, Product_Name, Sub_category_ID) \
                SELECT DISTINCT ON (Product_ID, Product_Name) Product_ID, Product_Name, Sub_category_ID \
                FROM stg.orders as ord \
                INNER JOIN blm.categorys as cat on ord.SubCategory = cat.Sub_category;")
connect.commit() 

In [33]:
# проверка записи данных в таблицу 'products'
cursor.execute("SELECT COUNT(*) FROM blm.products;")
cursor.fetchone()

(1894,)

In [34]:
# проверка записи данных в таблицу 'products'
cursor.execute("SELECT COUNT(DISTINCT(Product_ID, Product_Name))  \
                FROM stg.orders;")
cursor.fetchone()

(1894,)

### 3.3 Таблица сегменты - 'Segments'

In [35]:
# данные для записи в таблицу сегменты - 'segments'
cursor.execute("SELECT DISTINCT(Segment) FROM stg.orders;")
cursor.fetchall()

[('Consumer',), ('Corporate',), ('Home Office',)]

In [36]:
# запись данных в таблицу сегменты - 'segments'
cursor.execute("INSERT INTO blm.segments( \
                Segment) \
                SELECT DISTINCT(Segment) FROM stg.orders;")
connect.commit() 

In [37]:
# проверка записи данных в таблицу 'segments'
cursor.execute("SELECT * FROM blm.segments;")
cursor.fetchall()

[(1, 'Consumer'), (2, 'Corporate'), (3, 'Home Office')]

### 3.4 Таблица покупатели - 'Customers'

In [38]:
# проверка уникальности Product_ID
cursor.execute("SELECT Customer_ID, Customer_Name \
                FROM stg.orders \
                WHERE Customer_ID IN (SELECT Customer_ID FROM \
                (SELECT Customer_ID, COUNT(DISTINCT(Customer_Name)) \
                FROM stg.orders \
                GROUP BY Customer_ID \
                HAVING COUNT(DISTINCT(Customer_Name))>1) t) \
                GROUP BY Customer_ID, Customer_Name \
                ORDER BY Customer_ID;")
cursor.fetchmany(10)

[]

**Customer_ID** уникальны и соответствуют своему **Customer_Name**

In [39]:
# данные для записи в таблицу покупатели - 'customers'
cursor.execute("SELECT DISTINCT ON (Customer_ID, Customer_Name) Customer_ID, Customer_Name, Segment_ID \
                FROM stg.orders as ord \
                INNER JOIN blm.segments as seg on ord.Segment = seg.Segment;")
cursor.fetchmany(3)

[('AA-10315', 'Alex Avila', 1),
 ('AA-10375', 'Allen Armold', 1),
 ('AA-10480', 'Andrew Allen', 1)]

In [40]:
# запись данных в таблицу покупатели - 'customers'
cursor.execute("INSERT INTO blm.customers( \
                Customer_ID, Customer_Name, Segment_ID) \
                SELECT DISTINCT ON (Customer_ID, Customer_Name) Customer_ID, Customer_Name, Segment_ID \
                FROM stg.orders as ord \
                INNER JOIN blm.segments as seg on ord.Segment = seg.Segment;")
connect.commit() 

In [41]:
# проверка записи данных в таблицу 'customers'
cursor.execute("SELECT COUNT(*) FROM blm.customers;")
cursor.fetchone()

(793,)

In [42]:
# проверка записи данных в таблицу 'customers'
cursor.execute("SELECT COUNT(DISTINCT(Customer_ID, Customer_Name))  \
                FROM stg.orders;")
cursor.fetchone()

(793,)

### 3.5 Таблица класс доставки - 'Ship Modes'

In [43]:
# данные для записи в таблицу класс доставки - 'ship_modes'
cursor.execute("SELECT DISTINCT(Ship_mode) FROM stg.orders;")
cursor.fetchall()

[('Standard Class',), ('Second Class',), ('Same Day',), ('First Class',)]

In [44]:
# запись данных в таблицу класс доставки - 'ship_modes'
cursor.execute("INSERT INTO blm.ship_modes( \
                Ship_mode) \
                SELECT DISTINCT(Ship_mode) FROM stg.orders;")
connect.commit() 

In [45]:
# проверка записи данных в таблицу 'ship_modes'
cursor.execute("SELECT * FROM blm.ship_modes;")
cursor.fetchall()

[(1, 'Standard Class'),
 (2, 'Second Class'),
 (3, 'Same Day'),
 (4, 'First Class')]

### 3.6 Таблица регионы - 'Regions'

In [46]:
# данные для записи в таблицу регионы - 'regions'
cursor.execute("SELECT DISTINCT(Region) FROM stg.orders;")
cursor.fetchall()

[('South',), ('West',), ('East',), ('Central',)]

In [47]:
# запись данных в таблицу регионы - 'regions'
cursor.execute("INSERT INTO blm.regions( \
                Region) \
                SELECT DISTINCT(Region) FROM stg.orders;")
connect.commit() 

In [48]:
# проверка записи данных в таблицу 'regions'
cursor.execute("SELECT * FROM blm.regions;")
cursor.fetchall()

[(1, 'South'), (2, 'West'), (3, 'East'), (4, 'Central')]

### 3.7 Таблица менеджеры - 'Persons'

In [49]:
# данные для записи в таблицу менеджеры - 'persons'
cursor.execute("SELECT Person, Region_ID \
                FROM stg.people ppl \
                INNER JOIN blm.regions as reg on ppl.Region = reg.Region;")
cursor.fetchall()

[('Anna Andreadi', 2),
 ('Chuck Magee', 3),
 ('Kelly Williams', 4),
 ('Cassandra Brandow', 1)]

In [50]:
# запись данных в таблицу менеджеры - 'persons'
cursor.execute("INSERT INTO blm.persons( \
                Person, Region_ID) \
                SELECT Person, Region_ID \
                FROM stg.people ppl \
                INNER JOIN blm.regions as reg on ppl.Region = reg.Region;")
connect.commit()

In [51]:
# проверка записи данных в таблицу 'persons'
cursor.execute("SELECT * FROM blm.persons;")
cursor.fetchall()

[(1, 'Anna Andreadi', 2),
 (2, 'Chuck Magee', 3),
 (3, 'Kelly Williams', 4),
 (4, 'Cassandra Brandow', 1)]

### 3.8 Таблица адреса - 'Geos'

In [52]:
# проверка уникальности Postal_Code
cursor.execute("SELECT Postal_Code, State, City \
                FROM stg.orders \
                WHERE Postal_Code IN (SELECT Postal_Code FROM \
                (SELECT Postal_Code, COUNT(DISTINCT(State, City)) \
                FROM stg.orders \
                GROUP BY Postal_Code \
                HAVING COUNT(DISTINCT(State, City))>1) t) \
                GROUP BY Postal_Code, State, City \
                ORDER BY Postal_Code;")
cursor.fetchmany(10)

[(92024, 'California', 'Encinitas'), (92024, 'California', 'San Diego')]

Почтовому индексу **92024** соответствует **2 (два)** города - **'Encinitas'** и **'San Diego'**

In [53]:
# данные для записи в таблицу адреса - 'geos'
cursor.execute("SELECT DISTINCT ON (State, City) State, City, Country, Region_ID, Postal_Code \
                FROM stg.orders ord \
                INNER JOIN blm.regions as reg on ord.Region = reg.Region;")
cursor.fetchmany(5)

[('Alabama', 'Auburn', 'United States', 1, 36830),
 ('Alabama', 'Decatur', 'United States', 1, 35601),
 ('Alabama', 'Florence', 'United States', 1, 35630),
 ('Alabama', 'Hoover', 'United States', 1, 35244),
 ('Alabama', 'Huntsville', 'United States', 1, 35810)]

In [54]:
# запись данных в таблицу адреса - 'geos'
cursor.execute("INSERT INTO blm.geos( \
                State, City, Country, Region_ID, Postal_Code) \
                SELECT DISTINCT ON (State, City) State, City, Country, Region_ID, Postal_Code \
                FROM stg.orders ord \
                INNER JOIN blm.regions as reg on ord.Region = reg.Region;")
connect.commit()

In [55]:
# проверка записи данных в таблицу 'geos'
cursor.execute("SELECT COUNT(*) FROM blm.geos;")
cursor.fetchone()

(604,)

In [56]:
# проверка записи данных в таблицу 'geos'
cursor.execute("SELECT COUNT(DISTINCT(State, City))  \
                FROM stg.orders;")
cursor.fetchone()

(604,)

### 3.9 Таблица заказы - 'Orders'

In [57]:
# проверка уникальности Order_ID
cursor.execute("SELECT COUNT(Order_ID), COUNT(DISTINCT(Order_ID)) \
                FROM stg.orders;")
cursor.fetchone()

(9993, 5009)

**Order_ID** в таблице **'stg.orders'** не является уникальным  так как в составе заказа может быть несколько товаров

In [70]:
# данные для записи в таблицу заказы - 'orders'
cursor.execute("SELECT ord.Order_ID, ord.Order_Date, ord.Ship_Date, \
                       shm.Ship_mode_ID, cst.Customer_ID, gs.Geo_ID, prd.Row_ID, \
                       ord.Sales, ord.Quantity, ord.Discount, ord.Profit, \
                       COALESCE(NULLIF(rt.Returned, ''), 'No'), prs.Person_ID \
                FROM stg.orders ord \
                INNER JOIN blm.ship_modes AS shm ON ord.Ship_mode = shm.Ship_mode \
                INNER JOIN blm.customers AS cst ON ord.Customer_Name = cst.Customer_Name \
                INNER JOIN blm.geos AS gs ON ord.State = gs.State AND ord.City = gs.City \
                INNER JOIN blm.products AS prd ON ord.Product_Name = prd.Product_Name AND ord.Product_ID = prd.Product_ID \
                LEFT JOIN stg.returns AS rt ON ord.Order_ID = rt.Order_ID \
                INNER JOIN blm.persons AS prs ON gs.Region_ID = prs.Region_ID;")
cursor.fetchmany(3)

[('CA-2019-145779',
  datetime.date(2019, 5, 6),
  datetime.date(2019, 5, 10),
  1,
  'DB-13615',
  20,
  1182,
  Decimal('84.4160'),
  4,
  Decimal('0.20'),
  Decimal('27.4351999999999950'),
  'No',
  1),
 ('CA-2017-113215',
  datetime.date(2017, 9, 3),
  datetime.date(2017, 9, 8),
  1,
  'CP-12085',
  20,
  245,
  Decimal('238.1520'),
  3,
  Decimal('0.20'),
  Decimal('89.3069999999999900'),
  'No',
  1),
 ('CA-2017-147816',
  datetime.date(2017, 9, 24),
  datetime.date(2017, 9, 26),
  2,
  'CM-11935',
  20,
  1831,
  Decimal('35.1200'),
  2,
  Decimal('0.20'),
  Decimal('13.1700000000000000'),
  'No',
  1)]

In [67]:
# запись данных в таблицу заказы - 'orders'
cursor.execute("INSERT INTO blm.orders( \
                Order_ID, Order_Date, Ship_Date, Ship_mode_ID, Customer_ID, Geo_ID, Row_ID, \
                Sales, Quantity, Discount, Profit, Returned, Person_ID) \
                SELECT ord.Order_ID, ord.Order_Date, ord.Ship_Date, \
                       shm.Ship_mode_ID, cst.Customer_ID, gs.Geo_ID, prd.Row_ID, \
                       ord.Sales, ord.Quantity, ord.Discount, ord.Profit, \
                       COALESCE(NULLIF(rt.Returned, ''), 'No'), prs.Person_ID \
                FROM stg.orders ord \
                INNER JOIN blm.ship_modes AS shm ON ord.Ship_mode = shm.Ship_mode \
                INNER JOIN blm.customers AS cst ON ord.Customer_Name = cst.Customer_Name \
                INNER JOIN blm.geos AS gs ON ord.State = gs.State AND ord.City = gs.City \
                INNER JOIN blm.products AS prd ON ord.Product_Name = prd.Product_Name AND ord.Product_ID = prd.Product_ID \
                LEFT JOIN stg.returns AS rt ON ord.Order_ID = rt.Order_ID \
                INNER JOIN blm.persons AS prs ON gs.Region_ID = prs.Region_ID;")
connect.commit()

In [68]:
# проверка записи данных в таблицу 'orders'
cursor.execute("SELECT COUNT(*) \
                FROM blm.orders;")
cursor.fetchone()

(9993,)

In [69]:
# проверка записи данных в таблицу 'orders'
cursor.execute("SELECT COUNT(Returned) \
                FROM stg.orders;")
cursor.fetchone()

(9993,)

In [72]:
# закрытие подключения к базе данных 'postgres'
connect.close()